1. 简单线性回归示例

1. 导入库

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

2. 生成数据集

In [ ]:
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000)

3. 读取数据集

In [ ]:
## 构建一个数据迭代器
def load_array(data_arrays, batch_size, is_train = True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle = is_train)

## 设置每个batch的大小为10个样本，然后选择打乱数据
batch_size = 10
data_iter = load_array((features, labels), batch_size)
next(iter(data_iter))

[tensor([[-0.5844, -0.0526],
         [-1.0501, -1.0408],
         [-2.5948, -0.7348],
         [ 1.3322, -0.6363],
         [ 0.5904,  0.1136],
         [ 0.9053, -1.0497],
         [-0.2112, -0.7498],
         [-0.1890, -1.2082],
         [-1.3734,  0.4653],
         [ 1.3243, -0.4418]]),
 tensor([[ 3.2037],
         [ 5.6269],
         [ 1.5002],
         [ 9.0308],
         [ 4.9881],
         [ 9.5867],
         [ 6.3261],
         [ 7.9383],
         [-0.1250],
         [ 8.3471]])]

4. 定义模型

In [ ]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))  ## 输入特征数为2，输出标签为1

torch.nn.Sequential是PyTorch中一个模块容器（Module Container），用于按顺序堆叠多个神经网络层，简化模型定义的过程。
Sequential 让我们可以不用单独定义 forward() 方法，按顺序执行层操作。
适用于线性堆叠的网络结构（如 CNN、MLP）。
单层网络架构称为全连接层，在Linear类中定义

5. 初始化模型参数(w, b)

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

这里对第一层net的参数weight和bias进行初始化。通过weight.data和bias.data访问参数，通过nomal_和fill_方法替换参数。

6. 定义损失函数

In [6]:
## 均方误差（平方L2范数）计算，返回所有样本损失的平均值
loss = nn.MSELoss()

7. 定义优化算法

In [7]:
trainer = torch.optim.SGD(net.parameters(), lr = 0.03)

这行代码的作用是创建一个随机梯度下降（Stochastic Gradient Descent, SGD）优化器，用于更新 net（神经网络）模型的参数。
net.parameters()返回模型中所有的参数(w, b)，lr是learning rate。

8. 训练模型

In [ ]:
num_epochs = 3 ## 设置对数据集进行三轮的训练，每一轮遍历整个数据集
for epoch in range(num_epochs):
    for X, y in data_iter:
        l = loss(net(X), y) ## 计算损失
        trainer.zero_grad() ## 清空上一个epoch的梯度
        l.backward() ## 反向传播计算梯度
        trainer.step() ## 更新参数
    l = loss(net(features), labels) ## 计算整个数据集的损失
    print(f'epoch {epoch + 1}, loss {l:f}')

epoch 1, loss 0.000193
epoch 2, loss 0.000103
epoch 3, loss 0.000104


In [18]:
## 访问参数的梯度
print("weight gradient:", net[0].weight.grad)
print("bias gradient:", net[0].bias.grad)

weight gradient: tensor([[-0.0005,  0.0041]])
bias gradient: tensor([0.0019])
